In [5]:
import os
import os.path
import pickle
import re
import sys
import csv
import re
type_list = ['int', 'char', 'float', 'double', 'bool', 'void', 'short', 'long', 'signed', 'struct']

valid_ext = [".c", ".cpp"]

re_removed = "(\'.*\')|(\".*\")|(/\*.*\*/)|(//.*)"

keyword_set = set(type_list + ['sizeof'])

def pickle_dump(root_path, data, file_name):
    os.chdir(root_path)
    fp = open(file_name, "w")
    pickle.dump(data, fp)
    fp.close()

def pickle_load(root_path, file_name):
    os.chdir(root_path)
    fp_case = open(file_name, "r")
    dict_case = pickle.load(fp_case)
    fp_case.close()
    return dict_case


def is_valid_name(name):
    if re.match("[a-zA-Z_][a-zA-Z0-9_]*", name) == None:
        return False
    if name in keyword_set:
        return False
    return True

def is_func(line):
#int, __int64, void, char*, char *, struct Node, long long int, (void *)
#int func(int a, int *b, (char *) c)
    line = line.strip()
    if len(line) < 2:
        return None
# Rule 1: assume the function name line must ends with ) or {;
#    if line[-1] != ')' and line[-1] != '{':
#        return None
# Rule 2: (*) must in
    if '(' not in line: #or ')' not in line:
        return None
# Rule 3: # stands for #include or other primitives; / start a comment
    if line[0] == '#' or line[0] == '/':
        return None
# Rule 4: ends with ;
    if line.endswith(';'):
        return None
    if line.startswith('static'):
        line = line[len('static'):]
# replace pointer * and & as space
    line = re.sub('\*', ' ', line)
    line = re.sub('\&', ' ', line)


# replace '(' as ' ('
    #re.sub('(', ' ( ', line)
    line = re.sub('\(', ' \( ', line)
    line_split = line.split()

    if len(line_split) < 2:
        return None

    bracket_num = 0
    for ch in line:
        if ch == '(':
            bracket_num += 1

    has_type = False
    for type_a in type_list:
        if type_a in line_split[0]:
            has_type = True
#    if has_type == False:
#        return None
#    print line
    if bracket_num == 1:
        for index in range(len(line_split)):
            if '(' in line_split[index]:
                return line_split[index - 1]
    else:
        line = re.sub('\(', ' ', line)
        line = re.sub('\)', ' ', line)
        line_split = line.split()
        index = 0
        for one in line_split:
            if is_valid_name(one):
                index += 1
                if index == 2:
                    return one
        return None

def get_line_type(line):
    line = line.strip()
    if line.startswith("/*"):
#        print line
        return "comment_paragraph"
    elif line.startswith("//"):
        return "comment_line"
    elif line.startswith("#"):
        return "macro"
    return "other"

In [2]:
line_test='_gcry_aes_armv8_ce_cfb_enc (RIJNDAEL_context *ctx, unsigned char *iv,'

In [6]:
is_func(line_test)

'_gcry_aes_armv8_ce_cfb_enc'

In [29]:
def is_comment_end(line):
    #print line
    line = line.strip()
    if line.endswith('*/'):
        return True
    return False

def is_func_end(line, left_brack_num):
    line = line.strip()
    left_brack_num += line.count("{")
    if "}" in line:
        left_brack_num -= line.count("}")
        if left_brack_num == 0:
            return True
    return False

def func_name_extract(file_path):

    if not os.path.isfile(file_path):
        return


    file_list = []
    with open(file_path, "r", encoding='UTF8') as fp:
         for line in fp.readlines():
                 file_list.append(line)

    func_list = []

    i = -1
    while i < len(file_list) - 1:
        i += 1
        line = file_list[i]
        line_type = get_line_type(line)
        if line_type == "comment_line" or line_type == "macro":
            continue
        elif line_type == "comment_paragraph":
            while not is_comment_end(file_list[i]):
                i += 1
        else:
            line = re.sub(re_removed, "", line)
            if len(line) == 0:
                continue
            func_name = is_func(line)
            if func_name != None:
#                print i, func_name
                start_line = i
                left_brack_num = 0
                effective_line = 1
                while True and i < len(file_list):
#                    print i
#                    print line
                    line = (file_list[i]).strip()
                    line_type = get_line_type(line)
                    if line_type == "comment_line":
                        continue

#                    elif line_type == "comment_paragraph":
#                        continue
#                        while not is_comment_end(file_list[i]):
#                            i += 1
                    left_brack_num += line.count('{')
                    effective_line += 1
                    if "}" in line:
                        left_brack_num -= line.count("}")
                        if left_brack_num < 1:
                            break
#                    print left_brack_num
                    i += 1
                end_line = i
               # if func_name != None:
                func_list.append([file_path, func_name, start_line + 1, end_line + 1, end_line - start_line + 1])
#                print func_name
    return func_list

In [13]:
func_name_extract(r'C:\Users\hadri\Documents\GitHub\crypto_code_detection\data\crypto-library\files\libgcrypt\rijndael-armv8-ce.c')

[['C:\\Users\\hadri\\Documents\\GitHub\\crypto_code_detection\\data\\crypto-library\\files\\libgcrypt\\rijndael-armv8-ce.c',
  '_gcry_aes_enc_armv8_ce',
  43,
  223,
  181],
 ['C:\\Users\\hadri\\Documents\\GitHub\\crypto_code_detection\\data\\crypto-library\\files\\libgcrypt\\rijndael-armv8-ce.c',
  '_gcry_aes_armv8_ce_prepare_decryption',
  227,
  264,
  38],
 ['C:\\Users\\hadri\\Documents\\GitHub\\crypto_code_detection\\data\\crypto-library\\files\\libgcrypt\\rijndael-armv8-ce.c',
  '_gcry_aes_armv8_ce_encrypt',
  267,
  274,
  8],
 ['C:\\Users\\hadri\\Documents\\GitHub\\crypto_code_detection\\data\\crypto-library\\files\\libgcrypt\\rijndael-armv8-ce.c',
  '_gcry_aes_armv8_ce_decrypt',
  277,
  284,
  8],
 ['C:\\Users\\hadri\\Documents\\GitHub\\crypto_code_detection\\data\\crypto-library\\files\\libgcrypt\\rijndael-armv8-ce.c',
  '_gcry_aes_armv8_ce_cbc_enc',
  287,
  296,
  10],
 ['C:\\Users\\hadri\\Documents\\GitHub\\crypto_code_detection\\data\\crypto-library\\files\\libgcrypt\\ri

In [14]:
import os
entries = os.listdir(r'C:\Users\hadri\Documents\GitHub\crypto_code_detection\data\crypto-library\files\libgcrypt')

In [15]:
entries

['arcfour.c',
 'asm-common-aarch64.h',
 'asm-common-amd64.h',
 'asm-poly1305-aarch64.h',
 'asm-poly1305-amd64.h',
 'bithelp.h',
 'blake2.c',
 'blowfish.c',
 'bufhelp.h',
 'camellia-glue.c',
 'camellia.c',
 'camellia.h',
 'cast5.c',
 'chacha20-ppc.c',
 'chacha20.c',
 'cipher-aeswrap.c',
 'cipher-cbc.c',
 'cipher-ccm.c',
 'cipher-cfb.c',
 'cipher-cmac.c',
 'cipher-ctr.c',
 'cipher-eax.c',
 'cipher-gcm-intel-pclmul.c',
 'cipher-gcm.c',
 'cipher-internal.h',
 'cipher-ocb.c',
 'cipher-ofb.c',
 'cipher-poly1305.c',
 'cipher-selftest.c',
 'cipher-selftest.h',
 'cipher-xts.c',
 'cipher.c',
 'crc-armv8-ce.c',
 'crc-intel-pclmul.c',
 'crc-ppc.c',
 'crc.c',
 'des.c',
 'dsa-common.c',
 'dsa.c',
 'ecc-common.h',
 'ecc-curves.c',
 'ecc-ecdh.c',
 'ecc-ecdsa.c',
 'ecc-eddsa.c',
 'ecc-gost.c',
 'ecc-misc.c',
 'ecc.c',
 'elgamal.c',
 'gost-s-box.c',
 'gost28147.c',
 'gostr3411-94.c',
 'hash-common.c',
 'hash-common.h',
 'hmac-tests.c',
 'idea.c',
 'kdf.c',
 'keccak.c',
 'keccak_permute_32.h',
 'keccak_p

In [33]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [56]:
parsed_results={}
MY_DIR=r"C:\Users\hadri\Documents\GitHub"
FILE_DIR=r"crypto_code_detection\data\crypto-library\files"
library_to_parse=r'libgcrypt'
for e in entries :
    if e[-1]=='h' : continue
    print("considering : {}".format(e))
    n=np.array(func_name_extract(r'{}\{}\{}\{}'.format(MY_DIR,FILE_DIR,library_to_parse,e)))   
#     k="{}\{}".format(FILE_DIR,library_to_parse)
    parsed_results["{}\{}\{}".format(FILE_DIR,library_to_parse,e)]=n[:,1:]
#     f.write(str(n)+'\n')
# f.close()
save_obj(parsed_results, ''+library_to_parse+"_parsed_func_names" )

considering : arcfour.c
considering : blake2.c
considering : blowfish.c
considering : camellia-glue.c
considering : camellia.c
considering : cast5.c
considering : chacha20-ppc.c
considering : chacha20.c
considering : cipher-aeswrap.c
considering : cipher-cbc.c
considering : cipher-ccm.c
considering : cipher-cfb.c
considering : cipher-cmac.c
considering : cipher-ctr.c
considering : cipher-eax.c
considering : cipher-gcm-intel-pclmul.c
considering : cipher-gcm.c
considering : cipher-ocb.c
considering : cipher-ofb.c
considering : cipher-poly1305.c
considering : cipher-selftest.c
considering : cipher-xts.c
considering : cipher.c
considering : crc-armv8-ce.c
considering : crc-intel-pclmul.c
considering : crc-ppc.c
considering : crc.c
considering : des.c
considering : dsa-common.c
considering : dsa.c
considering : ecc-curves.c
considering : ecc-ecdh.c
considering : ecc-ecdsa.c
considering : ecc-eddsa.c
considering : ecc-gost.c
considering : ecc-misc.c
considering : ecc.c
considering : elgamal

In [57]:
parsed_results

{'crypto_code_detection\\data\\crypto-library\\files\\libgcrypt\\arcfour.c': array([['defined', '37', '48', '12'],
        ['_gcry_arcfour_amd64', '50', '58', '9'],
        ['do_encrypt_stream', '68', '114', '47'],
        ['encrypt_stream', '117', '123', '7'],
        ['do_arcfour_setkey', '129', '170', '42'],
        ['arcfour_setkey', '173', '180', '8'],
        ['selftest', '184', '207', '24'],
        ['sizeof', '213', '215', '3']], dtype='<U99'),
 'crypto_code_detection\\data\\crypto-library\\files\\libgcrypt\\blake2.c': array([['HAVE_COMPATIBLE_GCC_AMD64_PLATFORM_AS', '36', '74', '39'],
        ['unsigned', '137', '147', '11'],
        ['blake2_write', '158', '201', '44'],
        ['blake2b_set_lastblock', '204', '207', '4'],
        ['blake2b_is_lastblock', '209', '212', '4'],
        ['blake2b_increment_counter', '214', '218', '5'],
        ['rotr64', '220', '223', '4'],
        ['blake2b_transform_generic', '225', '341', '117'],
        ['_gcry_blake2b_transform_amd64_avx2', 